# 1. Setup Environment

In [203]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import statistics as stats
import seaborn as sns

from skopt.sampler import Lhs
from skopt.space import Space

%matplotlib inline

pd.options.display.max_rows = 200
pd.options.display.max_columns = None

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 2. Simulation Inputs

### 2.1. General Settings

In [204]:
sample_size = 20000 # number of unique simulations
run_time = 50 # number of years in each simulation
parameter_count = 8 # number of survival parameters in the algorithm

### 2.2. Survival Parameters
- Each parameter is assigned a range from which values will be sampled

In [205]:
parameter_labels = ['ClutchSize', 'FledgeRate', 'FledglingSurvival', 'InitialAbundance', 'JuvenileSurvival', 'AdultSurvival',
                    'SYBreeding', 'ASYBreeding']

In [206]:
# known
lower_ClutchSize, upper_ClutchSize = 4.137, 4.572
lower_FledgeRate, upper_FledgeRate = 0.343, 0.379
lower_FledglingSurvival, upper_FledglingSurvival = 0.586, 0.648
lower_InitialAbundance, upper_InitialAbundance = 2660000., 2940000.  

# predicted
lower_JuvenileSurvival, upper_JuvenileSurvival = 0.532, 0.588  # 0.56
lower_AdultSurvival, upper_AdultSurvival = 0.641, 0.709  # 0.675
lower_SYBreeding, upper_SYBreeding = 0.855, 0.945
lower_ASYBreeding, upper_ASYBreeding = 0.9025, 0.9975

In [207]:
raw_parameter_range = [[lower_ClutchSize, upper_ClutchSize], [lower_FledgeRate, upper_FledgeRate],
                       [lower_FledglingSurvival, upper_FledglingSurvival], [lower_InitialAbundance, upper_InitialAbundance],
                       [lower_JuvenileSurvival, upper_JuvenileSurvival], [lower_AdultSurvival, upper_AdultSurvival],
                       [lower_SYBreeding, upper_SYBreeding], [lower_ASYBreeding, upper_ASYBreeding]]

### 2.3. Environmental Stochasticity
- Year to year variation that is unique to each simulation

In [208]:
stochasticity_labels = ['ClutchSize', 'FledgeRate', 'FledglingSurvival', 'JuvenileSurvival', 'AdultSurvival',
                        'SYBreeding', 'ASYBreeding']

In [209]:
stoch_ClutchSize = 0.046
stoch_FledgeRate = 0.011
stoch_FledglingSurvival = 0.015
stoch_JuvenileSurvival = 0.015
stoch_AdultSurvival = 0.010
stoch_SYBreeding = 0.010
stoch_ASYBreeding = 0.015

In [210]:
stochasticity_values = [stoch_ClutchSize, stoch_FledgeRate, stoch_FledglingSurvival, stoch_JuvenileSurvival, 
                        stoch_AdultSurvival, stoch_SYBreeding, stoch_ASYBreeding]

In [211]:
stochasticity_dict = {a: b for (a, b) in zip(stochasticity_labels, stochasticity_values)}
stochasticity = pd.DataFrame.from_dict([stochasticity_dict], orient='columns')

# 3. Parameter Sets

### 3.1. Import LHS
- Optionally import LHS results and skip 3.2 - 3.3

In [212]:
random_values_df = pd.read_csv(r'C:\Users\Work\Desktop\MS Pub\GSA\LHS_20k8p.csv')
random_values_df = random_values_df.sample(sample_size).reset_index(drop=True)

In [213]:
random_values_df.rename(columns={'0': 'ClutchSize', '1': 'FledgeRate', '2': 'FledglingSurvival', '3': 'InitialAbundance',
                                 '4': 'JuvenileSurvival', '5': 'AdultSurvival', '6': 'SYBreeding', '7': 'ASYBreeding'},
                        inplace=True)

### 3.2. LHS Sampling
- Random values to simulate parameter uncertainty

In [214]:
# dimensions = [(0., 1.)] * parameter_count

# lhs = Lhs(lhs_type='classic', criterion='maximin', iterations=1000)
# random_values = lhs.generate(dimensions=dimensions, n_samples=sample_size)

### 3.3. Export LHS
- Optionally export LHS results

In [215]:
# random_values_df = pd.DataFrame(random_values)
# random_values_df.to_csv(r'C:\Users\Work\Desktop\MS Pub\GSA\LHS_20k8p.csv', index=False, header=True)

### 3.4. Generate Sets

In [216]:
def transformParameter(lower, upper, random):
    new_parameter = ((upper - lower) * random) + lower
    return new_parameter

In [217]:
parameter_sets = [[]] * sample_size
labeled_sets = [[]] * sample_size

for s in range(sample_size):
    parameter_sets[s] = [transformParameter(raw_parameter_range[p][0],
                                            raw_parameter_range[p][1],
                                            random_values_df.loc[s][p]) for p in range(parameter_count)]
    labeled_sets[s] = {a: b for (a, b) in zip(parameter_labels, parameter_sets[s])}
    
inputs = pd.DataFrame.from_dict(data=labeled_sets, orient='columns')

# 4. Simulation

### 4.1. Stable Age Distribution

In [218]:
sy_ratio = []
asy_ratio = []

In [219]:
def getStochasticity(params, stoch, labels):
    stoch_params = stoch.copy()
    for label in labels:
        stoch_params[label] = np.random.normal(params[label], stoch[label], 1)
    return stoch_params

In [220]:
for s in range(sample_size):
    
    params = inputs.iloc[s] # retrieve unique parameter set
    stoch = stochasticity.iloc[0] # retrieve stochasticity values
    labels = stochasticity_labels
    
    juv_count = []
    sy_count = []    
    asy_count = []
    population_size = []

    juv_count.append(0)
    sy_count.append(0)
    asy_count.append(params['InitialAbundance'])
    population_size.append(params['InitialAbundance'])
    
    delta = []
    y = 0
    
    while True:
        
        stoch_params = getStochasticity(params, stoch, labels) # apply year to year variation
           
        # growth algorithm
        breeding_pairs = ((stoch_params['SYBreeding'] * sy_count[y]) + (stoch_params['ASYBreeding'] * asy_count[y]))/2
        new_juv = breeding_pairs * stoch_params['ClutchSize'] * stoch_params['FledgeRate'] * stoch_params['FledglingSurvival']
        new_sy = new_juv * stoch_params['JuvenileSurvival']
        new_asy = (sy_count[y] + asy_count[y]) * stoch_params['AdultSurvival']

        juv_count.append(new_juv)
        sy_count.append(new_sy)
        asy_count.append(new_asy)
        population_size.append(new_sy + new_asy)

        if y > 0:
            delta.append(((sy_count[y-1]/population_size[y-1]) - (sy_count[y]/population_size[y])) + \
                        ((asy_count[y-1]/population_size[y-1]) - (asy_count[y]/population_size[y])))

        y += 1
        if y > 5: # minimum of 5 iterations
            if sum(delta[-3:]) < 0.01: # break when age distribution stabilizes
                break
        
        if y > 20:
            print("Warning: Age distribution stabilized slowly.")
    
    sy_ratio.append(sy_count[-1]/population_size[-1])
    asy_ratio.append(asy_count[-1]/population_size[-1])

In [221]:
stable = pd.DataFrame(list(zip(sy_ratio, asy_ratio)), columns=['%SY', '%ASY'])
trial = pd.DataFrame(list(zip(juv_count, sy_count, asy_count, population_size)), columns=['Juv', 'SY', 'ASY', 'Total'])

In [222]:
stable.sample(5)

,%SY,%ASY
12389,0.272289,0.727711
10566,0.277373,0.722627
4399,0.257924,0.742076
10836,0.261293,0.738707
8002,0.269065,0.730935


In [223]:
print('Mean Stable Age Distribution: {} % SY, {} % ASY'.format(round(np.mean(sy_ratio), 3) * 100,
                                                               round(np.mean(asy_ratio), 3) * 100))

Mean Stable Age Distribution: 27.3 % SY, 72.7 % ASY


### 4.2. Simulate Growth
- Initial abundance is distributed based on stable age distribution

In [224]:
start_point = []
end_point = []
growth_rate = []

In [225]:
for s in range(sample_size):

    params = inputs.loc[s] # retrieve unique parameter set
    stoch = stochasticity.iloc[0] # retrieve stochasticity values
    labels = stochasticity_labels
    
    juv_count = []
    sy_count = []    
    asy_count = []
    population_size = []
    annual_growth = []

    juv_count.append(0)
    sy_count.append(params['InitialAbundance'] * sy_ratio[s])
    asy_count.append(params['InitialAbundance'] * asy_ratio[s])
    population_size.append(params['InitialAbundance'])
    
    for y in range(run_time):
        
        stoch_params = getStochasticity(params, stoch, labels) # apply year to year variation
        
        # growth algorithm
        breeding_pairs = ((stoch_params['SYBreeding'] * sy_count[y]) + (stoch_params['ASYBreeding'] * asy_count[y]))/2
        new_juv = breeding_pairs * stoch_params['ClutchSize'] * stoch_params['FledgeRate'] * stoch_params['FledglingSurvival']
        new_sy = new_juv * stoch_params['JuvenileSurvival']
        new_asy = (sy_count[y] + asy_count[y]) * stoch_params['AdultSurvival']

        # store annual data
        juv_count.append(new_juv)
        sy_count.append(new_sy)
        asy_count.append(new_asy)
        population_size.append(new_sy + new_asy)
        annual_growth.append((population_size[y+1] - population_size[y]) / population_size[y])
    
    # store simulation data
    start_point.append(population_size[0])
    end_point.append(population_size[-1])
    growth_rate.append(np.mean(annual_growth) * 100)

In [226]:
outputs = pd.DataFrame(list(zip(start_point, end_point)), columns=['Input', 'Output'])
growth = pd.DataFrame(list(zip(juv_count, sy_count, asy_count, population_size)), columns=['Juv', 'SY', 'ASY', 'Total'])

In [227]:
outputs.head(5)
growth.tail(5)

,Input,Output
0,2.935815e+06,404874.500079
1,2.755107e+06,98354.763097
2,2.909077e+06,9699.965204
3,2.840050e+06,11265.240275
4,2.801774e+06,22156.306290


,Juv,SY,ASY,Total
46,20917.174531,11801.416832,30358.892395,42160.309227
47,20733.257798,12423.674600,27052.906865,39476.581465
48,19933.660012,11480.654868,26162.515576,37643.170444
49,16779.515901,9670.330426,24366.743047,34037.073473
50,17286.665804,9872.916210,21831.886493,31704.802703


In [228]:
print('Annual Mean Growth: {} %'.format(round(np.mean(growth_rate), 2)))

Annual Mean Growth: -7.08 %


### 4.3. Compile Results
- Optionally export the dataset

In [229]:
data = pd.concat([inputs, outputs['Output']], axis=1).reset_index()
data.rename(columns={'index': 'ID'}, inplace=True)

In [230]:
data.sample(5)

,ID,ClutchSize,FledgeRate,FledglingSurvival,InitialAbundance,JuvenileSurvival,AdultSurvival,SYBreeding,ASYBreeding,Output
13859,13859,4.352115,0.365299,0.602740,2.785221e+06,0.553559,0.659804,0.923872,0.917454,18142.024618
18488,18488,4.462546,0.356454,0.639301,2.868939e+06,0.576194,0.646692,0.856667,0.952474,40315.904735
6451,6451,4.545222,0.356290,0.616479,2.696451e+06,0.533400,0.707816,0.944071,0.992267,475292.265768
17951,17951,4.544594,0.367867,0.639327,2.842665e+06,0.546951,0.667902,0.869880,0.909078,98856.561464
7568,7568,4.551079,0.354403,0.635438,2.746052e+06,0.574290,0.690330,0.933440,0.949035,491022.117790


In [231]:
data.to_csv(r'C:\Users\Work\Desktop\MS Pub\GSA\grackle_20k8p_illinois.csv', header=True, index=False)